# Analyse des tweets

##### Import de la base nettoyée

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

#Tokenization 
import nltk
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize import TweetTokenizer
from functools import reduce
from operator import add
import collections

#Word2Vec
from gensim.models import Word2Vec
from multiprocessing import cpu_count

#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#BERT
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

#Clustering 
from sklearn.cluster import KMeans


#################################################
#################################################

# Import de la base intiale 
f = open('1377884570_tweet_global_warming.txt', 'r',newline='', encoding='ISO-8859-1')
content = f.read().split('\r')

content_new=[]
for x in content : 
    if len(x)>0:
        content_new.append(x)

content_new=content_new[1:len(content_new)]

#################################################
#################################################

# Création du dataframe

col_tweet=[]
col_existence=[]
col_score=[]

#Split tweet , Note

for line in content_new:
    if len(line.split('[link]'))==2:
        (x,y)=line.split('[link]')
        col_tweet.append(x)
        col_existence.append(y)
    else : 
        if len(line.split(',Yes,'))==2:
            col_tweet.append(line.split(',Yes,')[0])
            col_existence.append(',Yes,'+line.split(',Yes,')[1])
        elif len(line.split(',No,'))==2:
            col_tweet.append(line.split(',No,')[0])
            col_existence.append(',No,'+line.split(',No,')[1])
        elif len(line.split(',Y,'))==2:
            col_tweet.append(line.split(',Y,')[0])
            col_existence.append(',Yes,'+line.split(',Y,')[1])
        elif len(line.split(',N/A,'))==2:
            col_tweet.append(line.split(',N/A,')[0])
            col_existence.append(',N/A,'+line.split(',N/A,')[1])
        elif len(line.split(',NA,'))==2:
            col_tweet.append(line.split(',NA,')[0])
            col_existence.append(',NA,'+line.split(',NA,')[1])
        elif len(line.split(',N,'))==2:
            col_tweet.append(line.split(',N,')[0])
            col_existence.append(',No,'+line.split(',N,')[1])
        else : 
            print('erreur')
            #print(line.split('[link]'))
col_tweet.append('I truly  Fat ASS Gore should get the Scam Artist Award of the decade with his Global Warming and Energy Credits worth close to Billion')
col_existence.append(' ,NA')
col_tweet.append('Despite Climategate, LEFT investing heavily in global warming hysteria as new way 2 impose nat\'l & international controls on human freedom.')
col_existence.append(' ,NA')
        
# Split Existence/Note
col_existence_new=[]

for x in col_existence:
    if len(x.split(','))==3:
        col_existence_new.append(x.split(',')[1])
        col_score.append(x.split(',')[2])
    else:
        col_existence_new.append('NA')
        col_score.append('NA')
        
#Nettoyage existence
for avis in range(len(col_existence_new)):
    if col_existence_new[avis]=='NA' or col_existence_new[avis]=='N/A' or col_existence_new[avis]=='':
        col_existence_new[avis]=np.nan
        
#Nettoyage score
for score in range(len(col_score)):
    if 'NA' not in col_score[score]:
        col_score[score]=col_score[score].split('\t')[0]
        if len(col_score[score].split('"'))>1:
            col_score[score]=float(col_score[score].split('"')[0])
        else: 
            col_score[score]=float(col_score[score])
            
    else : 
        col_score[score]=np.nan

#Creation du DataFrame
dic={'Tweet':col_tweet,'Existence':col_existence_new,'Score':col_score}
df=pd.DataFrame(dic)


df.drop_duplicates(['Tweet'], inplace=True)
df.reset_index(inplace=True,drop=True)


#################################################
#################################################


#On prend toutes les phrases de touts les texts, et on les concatène dans une liste, en les traitant auparavant

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

def remove_hashtags(tokens):
    tokens= map(lambda x : x.replace('#',''),tokens) #map : parcours tout les tokens
    return list(tokens)

def remove_url(tokens): #pb pour https
    tokens= filter(lambda x: "http" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

def remove_html(tokens):
    tokens= filter(lambda x: x[0]+x[-1]!='<>',tokens)
    return list(tokens)

def remove_www(tokens):
    tokens= filter(lambda x: "www" not in x, tokens) #filter : garde là où il y a True
    return list(tokens)

'''
def remove_x95(clean_corpus):
    for sentence_r in range(len(clean_corpus)):
        sentence=clean_corpus[sentence_r]
        for x in range(len(sentence)):
            if '\x95' in sentence[x]:
                y=sentence[x].split('\x95')
                new_x=''
                for part_x in y:
                    new_x=new_x+part_x
                sentence[x]=new_x
        clean_corpus[sentence_r]=sentence
    return(clean_corpus)
'''

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def clean_ponctuation(text_tokens): # Nettoyage de la ponctuation

    list_word_clean_ponctuation=[]
    for tweet in text_tokens:
        list_tweet=[]
        for word in tweet:
            if len(word)<2:
                if (word=='a') or (word=='i') or (word=='u'):
                    list_tweet.append(word)
                if RepresentsInt(word):
                    list_tweet.append(word)
            else :
                if (word!='..') & (word!='...') & (word!='rt'):
                    list_tweet.append(word)
        list_word_clean_ponctuation.append(list_tweet)
    
    return(list_word_clean_ponctuation)

def remove_arobase(text_tokens):
    
    list_new_tokens=[]
    for tweet in text_tokens:
        new_tweet=[]
        for word in tweet: 
            if '@' not in word:
                new_tweet.append(word)
        list_new_tokens.append(new_tweet)
    
    return(list_new_tokens)

###############################################################
###############################################################
###############################################################

def clean_text_first(corpus):
    
    tok=TweetTokenizer()
    tokens=[]
    for sample in corpus:
        token=tok.tokenize(sample) 
        token=remove_url(token)
        token=remove_html(token)
        token=remove_hashtags(token)
        token=remove_www(token)
        token=list(map(lambda x : x.lower(),token)) #.lower() : met les majuscules en minuscules
        tokens.append(token) #ajout du token à l'ensemble des phrases
    
    #Nettoyage de la ponctuation
    tokens=clean_ponctuation(tokens)
    
    #Nettoyage des arobase : pour la plupart, se sont des noms propres
    tokens=remove_arobase(tokens)
    
    return tokens

def clean_text_second(corpus,threshold): #On rajoute l'association des mots qui vont ensembles
    
    #clean les textes
    tokens=clean_text_first(corpus)
    
    #associer les mots
    phrases=Phrases(tokens,threshold=threshold) #On fait apprendre le modèle d'association sur tout les mots
    phraser=Phraser(phrases) #Outil pour associer
    
    clean_tokens=[]
    for token in tokens: #On parcours les phrases et on associe les mots
        new_tokens=phraser[token]
        clean_tokens.append(new_tokens)
        
    #tokens = remove_x95(tokens)
    
    return(clean_tokens)



#################################################
#################################################

In [ ]:
print('On dispose de {} tweets.'.format(df.shape[0]))

print(' ')
print('On a {} données manquantes sur le label de l\'avis du tweet (Yes, No) .'.format(str(df.isnull().sum()['Existence'])))
print(' ')

#Personnes convaincues du changement climatique 
print('On a {} tweets qui croit au Changement climatique.'.format(str(df[df.Existence=='Yes'].shape[0])))
m=round(df[df.Existence=='Yes'].Score.describe()['mean'],2)
std=round(df[df.Existence=='Yes'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))
print(' ')

#Personnes qui doutent du changement climatique 
print('On a {} tweets qui remettent en doute le Changement climatique.'.format(str(df[df.Existence=='No'].shape[0])))
m=round(df[df.Existence=='No'].Score.describe()['mean'],2)
std=round(df[df.Existence=='No'].Score.describe()['std'],2)
print('Parmis ces personnes, le score est de {} en moyenne, avec un écart-type de {}.'.format(str(m),str(std)))

###### Première analyse des mots les plus importants

In [ ]:
# Bases des tweets Yes et des tweets No

df_yes=df[df.Existence=='Yes'].reset_index()
Tweet_clean_yes=clean_text_second(df_yes.Tweet,threshold=1000) #avec association de mots
Tweet_clean_yes=reduce(add, Tweet_clean_yes)

df_no=df[df.Existence=='No'].reset_index()
Tweet_clean_no=clean_text_second(df_no.Tweet,threshold=1000) #avec association de mots
Tweet_clean_no=reduce(add, Tweet_clean_no)

In [ ]:
# Mots les plus occurents dans la base No
counter=collections.Counter(Tweet_clean_no)

number_word=30
print('Les {} mots qui apparaissent le plus dans la base No sont (par ordre décroissant) :'.format(number_word))
print(' ')
for word in counter.most_common(number_word):
    print(word[0]+' ('+str(word[1])+') ') 

In [ ]:
# Mots les plus occurents dans la base Yes
counter=collections.Counter(Tweet_clean_yes)

number_word=30
print('Les {} mots qui apparaissent le plus dans la base Yes sont (par ordre décroissant) :'.format(number_word))
print(' ')
for word in counter.most_common(number_word):
    print(word[0]+' ('+str(word[1])+') ') 

Ils sont assez similaires. On regarde ceux qui font partis das 200 mots les plus occurents chez les Yes et qui ne font pas partis des 200 mots les plus occurents chez les No. Et inversement.

In [ ]:
counter=collections.Counter(Tweet_clean_yes)
list_commun_yes=[word[0] for word in counter.most_common(200)]

counter=collections.Counter(Tweet_clean_no)
list_commun_no=[word[0] for word in counter.most_common(200)]

list_spe_yes=[]
for word in list_commun_yes:
    if word not in list_commun_no:
        list_spe_yes.append(word)
        
list_spe_no=[]
for word in list_commun_no:
    if word not in list_commun_yes:
        list_spe_no.append(word)

list_spe_yes, list_spe_no

On s'interesse maintenant en détails aux statistiques de présences de ces mots

In [ ]:
mots_interessants_yes=['green','us','could','world','energy','earth','fight','worse','help','carbon']
mots_interessants_no=['gore','hoax','scam','climategate','gop','teaparty','palin','collapse','fraud','believe']

counter_yes=collections.Counter(Tweet_clean_yes)
counter_no=collections.Counter(Tweet_clean_no)

#Fréquence de ces mots
frequence_yes_in_yes={}
frequence_yes_in_no={}
for word in mots_interessants_yes:
    frequence_yes_in_yes[word]=counter_yes[word]/len(Tweet_clean_yes)*100
    frequence_yes_in_no[word]=counter_no[word]/len(Tweet_clean_no)*100
    
    
frequence_no_in_no={}
frequence_no_in_yes={}
for word in mots_interessants_no:
    frequence_no_in_no[word]=(counter_no[word]/len(Tweet_clean_no))*100
    frequence_no_in_yes[word]=(counter_yes[word]/len(Tweet_clean_yes))*100
    

In [ ]:
def Number_apparition_tweets(list_tweet,word):
    number=0
    for tweet in list_tweet:
        if word in tweet:
            number=number+1
    return(number)

In [ ]:
print('Mots intéressants en faveur du Changement climatique :')
print(' ')
print('fréquence : tout mots compris de la base Yes/No')
print(' ')
print('Fréquences dans les tweets Yes :')
print(' ')
list_tweet_yes=clean_text_second(df_yes.Tweet,threshold=1000)
for key in frequence_yes_in_yes.keys():
    print(key+': frequence = '+str(round(frequence_yes_in_yes[key],2))+'%'+' , nombre de tweets le contenant = '+str(Number_apparition_tweets(list_tweet_yes,key))+' (sur {})'.format(len(list_tweet_yes))+'--> {}%'.format(round(100*Number_apparition_tweets(list_tweet_yes,key)/len(list_tweet_yes),2)))                                   
print(' ') 
print('#'*10)
print(' ')
print('Fréquences dans les tweets No :')
print(' ')
list_tweet_no=clean_text_second(df_no.Tweet,threshold=1000)
for key in frequence_yes_in_no.keys():
    print(key+': frequence  = '+str(round(frequence_yes_in_no[key],2))+'%'+' , nombre de tweets le contenant = '+str(Number_apparition_tweets(list_tweet_no,key))+' (sur {})'.format(len(list_tweet_no))+'--> {}%'.format(round(100*Number_apparition_tweets(list_tweet_no,key)/len(list_tweet_no),2))) 


In [ ]:
print('Mots intéressants contre Changement climatique :')
print(' ')
print('fréquence : tout mots compris de la base Yes/No')
print(' ')
print('Fréquences dans les tweets No :')
print(' ')
list_tweet_no=clean_text_second(df_no.Tweet,threshold=1000)
for key in frequence_no_in_no.keys():
    print(key+': frequence = '+str(round(frequence_no_in_no[key],2))+'%'+' , nombre de tweets le contenant = '+str(Number_apparition_tweets(list_tweet_no,key))+' (sur {})'.format(len(list_tweet_no))+'--> {}%'.format(round(100*Number_apparition_tweets(list_tweet_no,key)/len(list_tweet_no),2)))                                   
print(' ') 
print('#'*10)
print(' ')
print('Fréquences dans les tweets No :')
print(' ')
list_tweet_yes=clean_text_second(df_yes.Tweet,threshold=1000)
for key in frequence_no_in_yes.keys():
    print(key+': frequence  = '+str(round(frequence_no_in_yes[key],2))+'%'+' , nombre de tweets le contenant = '+str(Number_apparition_tweets(list_tweet_yes,key))+' (sur {})'.format(len(list_tweet_yes))+'--> {}%'.format(round(100*Number_apparition_tweets(list_tweet_yes,key)/len(list_tweet_yes),2))) 


Commentaires....

##### Modèle LDA

In [ ]:
import gensim
import nltk
import time
from nltk.corpus import wordnet
import warnings


warnings.filterwarnings("ignore")
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer() 
warnings.filterwarnings("ignore")

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_return ={"j":wordnet.ADJ,
                 "n": wordnet.NOUN,
                 "v": wordnet.VERB,
                 "R": wordnet.ADV}
    return(tag_return.get(tag, wordnet.NOUN))

In [ ]:
#Représentation Bag-Of-Word

Tweet_clean=clean_text_second(df.Tweet,threshold=1000)

#####Etape déjà faite dans la nouvelle fonction clean_text
Tweet_clean_lemmatize=[]
for tweet in Tweet_clean:
    new_tweet=[]
    for token in tweet:
        if token!='':
            new_token=lemmatizer.lemmatize(token,get_wordnet_pos(token))
            if new_token=='warm': #traitement du warming
                new_token='warming'
            new_tweet.append(new_token)
    Tweet_clean_lemmatize.append(new_tweet)
Tweet_clean=Tweet_clean_lemmatize

# On enlève les mots de trois lettres ou moins 
Tweet_clean_lemmatize=[]
for tweet in Tweet_clean:
    new_tweet=[]
    for token in tweet:
        if len(token)>3:
            new_tweet.append(token)
    Tweet_clean_lemmatize.append(new_tweet)
Tweet_clean=Tweet_clean_lemmatize  

In [ ]:
# Représentation Bag-Of-Word 

def Representation_BOW(Tweet_clean):
    
    bow=pd.DataFrame([Tweet_clean]).T
    bow.columns=['headline_text']
    bow=bow.headline_text
    
    dictionary = gensim.corpora.Dictionary(bow)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in bow]
    
    return(bow_corpus)
    
# Sur la base entière
BOW_global=Representation_BOW(clean_text_second(df[(df.Existence=='Yes') | (df.Existence=='No')].Tweet,threshold=1000))
BOW_yes=Representation_BOW(clean_text_second(df[df.Existence=='Yes'].Tweet,threshold=1000))
BOW_no=Representation_BOW(clean_text_second(df[df.Existence=='No'].Tweet,threshold=1000))


In [ ]:
number_topics=3
print(' ')
# Construction du modèle LDA sur la base des Yes et No
print('Construction du modèle sur le base entière..')
t=time.time()
lda_model_global = gensim.models.LdaMulticore(BOW_global, num_topics=number_topics, id2word=dictionary, passes=2, workers=2)
print('DONE ({}s)'.format(str(round(time.time()-t,2))))
print(' ')

# Construction du modèle LDA sur la base des Yes
print('Construction du modèle sur le base Yes..')
t=time.time()
lda_model_yes = gensim.models.LdaMulticore(BOW_yes, num_topics=number_topics, id2word=dictionary, passes=2, workers=2)
print('DONE ({}s)'.format(str(round(time.time()-t,2))))
print(' ')

# Construction du modèle LDA sur la base des No
print('Construction du modèle sur le base No..')
t=time.time()
lda_model_no = gensim.models.LdaMulticore(BOW_no, num_topics=number_topics, id2word=dictionary, passes=2, workers=2)
print('DONE ({}s)'.format(str(round(time.time()-t,2))))


In [ ]:
list_model={'global':lda_model_global, 'Yes':lda_model_yes, 'No':lda_model_no}

# Pour chaque sujet remonté par le modèle, on regarde quels sont les mots qui apparaissent et avec quels poids
print(' ')
for model in list_model.keys():
    print('Sujets qui apparaissent dans la base {} : '.format(model))
    print(' ')
    for idx, topic in lda_model.print_topics(-1):
        print('Sujet numéro : {} \nWords: {}'.format(idx, topic))
        print(' ')
    print('#'*30)
    print(' ')

Il faut encore nettoyer certains mots 

##### Etude de la tonalité des tweets avec SentiWordNet

Pour chaque tweet, on prend les sentiments associés aux mots du tweet. On fait ensuite la moyenne des score. A la fin, pour chaque tweet on un score associé à la positivité, négativité et neutralité.

In [ ]:
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn
from nltk.stem import PorterStemmer
nltk.download('sentiwordnet')

In [ ]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    return None

def get_sentiment(word,tag):
    """ returns list of pos neg and objective score. But returns empty list if not present in senti wordnet. """

    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV):
        return []

    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    synsets = wordnet.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]


# Pour chaque tweet calcul des trois scores
def sentiments_score(Tweets,Initial_Tweet,labels):
    
    Tweet_sentiments=[]
    for tweet_index in range(len(Tweets)):

        list_tweet=[]

        # Append le tweet initial
        list_tweet.append(Initial_Tweet[tweet_index])

        # Append les scores

        ##Calcul des scores pour chaque mot du tweet
        pos_val = nltk.pos_tag(Tweets[tweet_index])
        senti_val = [get_sentiment(x,y) for (x,y) in pos_val]

        ## Moyenne de ces scores
        pos_score=0
        neg_score=0
        neutre_score=0
        size_senti=0
        for score in senti_val:
            if len(score)==3:
                pos_score=pos_score+score[0]
                neg_score=neg_score+score[1]
                neutre_score=neutre_score+score[2]
                size_senti=size_senti+1
        if size_senti==0:
            list_tweet.append('pas de mots connus par SWN')
        else:
            pos_score=pos_score/size_senti
            neg_score=neg_score/size_senti
            neutre_score=neutre_score/size_senti
        
            list_tweet.append([pos_score,neg_score,neutre_score])
        
        # Append le label 
        list_tweet.append(labels[tweet_index])
    
        Tweet_sentiments.append(list_tweet)
    
    return(Tweet_sentiments)
    

In [ ]:
# Import de la base des tweets et calcul des scores de sentiments

Tweet_clean=clean_text_second(df.Tweet,threshold=1000)
Labels=df.Existence

print(' ')
print('Calcul des scores de chaque mot et des scores moyens pour chaque tweet..')
t=time.time()
Base_SWN=sentiments_score(Tweet_clean,df.Tweet,Labels)
print('DONE ({}s)'.format(str(round(time.time()-t,2))))

In [ ]:
# Etude des résultats
Base_SWN=pd.DataFrame(Base_SWN,columns=['Tweet','Scores','Label'])

#############################################
#############################################

# Scores obtenus pour les Yes
Base_SWN_yes=Base_SWN[(Base_SWN.Label=='Yes')&(Base_SWN.Scores!='pas de mots connus par SWN')]

print(' Pour les Tweets qui croient au réchauffement climatique : ')
print(' ')

# Print des 10 premiers tweets 
for tweet_index in range(10):
    print(list(Base_SWN_yes.Tweet)[tweet_index])
    print('Scores (pos/neg/neutre) : '+str(list(Base_SWN_yes.Scores)[tweet_index]))
    print(' ')
#############################################
#############################################


print('#'*100)
print('#'*100)
print(' ')

print(' Pour les Tweets qui ne croient pas  au réchauffement climatique : ')
print(' ')
# Scores obtenus pour les No
Base_SWN_no=Base_SWN[(Base_SWN.Label=='No')&(Base_SWN.Scores!='pas de mots connus par SWN')]

# Print des 10 premiers tweets 
for tweet_index in range(10):
    print(list(Base_SWN_no.Tweet)[tweet_index])
    print('Scores (pos/neg/neutre) : '+str(list(Base_SWN_no.Scores)[tweet_index]))
    print(' ')

In [ ]:
# On peut scorer un Tweet en faisant le classant positif si son score négatif est plus grand
# que son score négatif ; et inversement. Si les deux scores sont égaux on le classe comme neutre

def nombre_pos_neg_neut(base):
    
    nombre_pos=np.sum([(list(base.Scores)[tweet][0]>list(base.Scores)[tweet][1])*1 for tweet in range(len(base))])
    nombre_neg=np.sum([(list(base.Scores)[tweet][1]>list(base.Scores)[tweet][0])*1 for tweet in range(len(base))])
    nombre_neut=np.sum([(list(base.Scores)[tweet][0]==list(base.Scores)[tweet][1])*1 for tweet in range(len(base))])

    return([nombre_pos,nombre_neg,nombre_neut])

# Sur toute la base 
data=Base_SWN[Base_SWN.Scores!='pas de mots connus par SWN']
results=nombre_pos_neg_neut(data)
print(' ')
print('Sur toute la base ({} tweets) : '.format(len(data)))
print(' ')
print('Le nombre de tweets jugés positifs est {} --> {}%'.format(str(results[0]),str(round(100*results[0]/len(data),2))))
print('Le nombre de tweets jugés négatifs est {} --> {}%'.format(str(results[1]),str(round(100*results[1]/len(data),2))))
print('Le nombre de tweets jugés neutres est {} --> {}%'.format(str(results[2]),str(round(100*results[2]/len(data),2))))
print(' ')
print('#'*30)
print('#'*30)
print(' ')

# Sur la bases composée des Yes  
data=Base_SWN_yes
results=nombre_pos_neg_neut(data)
print(' ')
print('Sur la base composée des Yes ({} tweets) : '.format(len(data)))
print(' ')
print('Le nombre de tweets jugés positifs est {} --> {}%'.format(str(results[0]),str(round(100*results[0]/len(data),2))))
print('Le nombre de tweets jugés négatifs est {} --> {}%'.format(str(results[1]),str(round(100*results[1]/len(data),2))))
print('Le nombre de tweets jugés neutres est {} --> {}%'.format(str(results[2]),str(round(100*results[2]/len(data),2))))
print(' ')
print('#'*30)
print('#'*30)
print(' ')

# Sur la bases composée des No 
data=Base_SWN_no
results=nombre_pos_neg_neut(data)
print(' ')
print('Sur la base composée des No ({} tweets) : '.format(len(data)))
print(' ')
print('Le nombre de tweets jugés positifs est {} --> {}%'.format(str(results[0]),str(round(100*results[0]/len(data),2))))
print('Le nombre de tweets jugés négatifs est {} --> {}%'.format(str(results[1]),str(round(100*results[1]/len(data),2))))
print('Le nombre de tweets jugés neutres est {} --> {}%'.format(str(results[2]),str(round(100*results[2]/len(data),2))))
print(' ')
print('#'*30)
print('#'*30)
print(' ')


In [ ]:
# Calculs de statistiques détaillés sur les scores 

def stats_scores(base):
    
    lis_pos=[list(base.Scores)[tweet][0] for tweet in range(len(list(base.Scores)))]
    list_neg=[list(base.Scores)[tweet][1] for tweet in range(len(list(base.Scores)))]
    list_neutre=[list(base.Scores)[tweet][2] for tweet in range(len(list(base.Scores)))]
    
    pos_score=np.mean(lis_pos)
    pos_score_ecart_type=np.std(lis_pos)
    pos_max=np.max(lis_pos)
    pos_3quant=np.quantile(lis_pos,0.75)
    dic_pos={'moyenne':round(pos_score,3),'std':round(pos_score_ecart_type,3),'max':round(pos_max,3),'quant_3':round(pos_3quant,3)}
    
    neg_score=np.mean(list_neg)
    neg_score_ecart_type=np.std(list_neg)
    neg_max=np.max(list_neg)
    neg_3quant=np.quantile(list_neg,0.75)
    dic_neg={'moyenne':round(neg_score,3),'std':round(neg_score_ecart_type,3),'max':round(neg_max,3),'quant_3':round(neg_3quant,3)}
    
    neut_score=np.mean(list_neutre)
    neut_score_ecart_type=np.std(list_neutre)
    neut_max=np.max(list_neutre)
    neut_3quant=np.quantile(list_neutre,0.75)
    dic_neut={'moyenne':round(neut_score,3),'std':round(neut_score_ecart_type,3),'max':round(neut_max,3),'quant_3':round(neut_3quant,3)}
    
    disc_results={'positifs':dic_pos, 'negatifs':dic_neg,'neutre':dic_neut}
    
    return(disc_results)

#Sur la base globale 
Base_SWN_scoreconnus=Base_SWN[Base_SWN.Scores!='pas de mots connus par SWN']
disc_results=stats_scores(Base_SWN_scoreconnus)

print(' ')
print(' Sur l\'ensemble de la base ({} tweets) : '.format(len(Base_SWN_scoreconnus)) )
print(' ')
print('Pour les scores positifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['positifs']['moyenne'],disc_results['positifs']['std'],disc_results['positifs']['max'],disc_results['positifs']['quant_3']))
print('Pour les scores negatifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['negatifs']['moyenne'],disc_results['negatifs']['std'],disc_results['negatifs']['max'],disc_results['negatifs']['quant_3']))
print('Pour les scores neutres : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['neutre']['moyenne'],disc_results['neutre']['std'],disc_results['neutre']['max'],disc_results['neutre']['quant_3']))
print(' ')
print('#'*30)
print(' ')

#Sur la base Yes
disc_results=stats_scores(Base_SWN_yes)

print(' ')
print(' Sur la bases des tweets Yes ({} tweets) : '.format(len(Base_SWN_yes)))
print(' ')
print('Pour les scores positifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['positifs']['moyenne'],disc_results['positifs']['std'],disc_results['positifs']['max'],disc_results['positifs']['quant_3']))
print('Pour les scores negatifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['negatifs']['moyenne'],disc_results['negatifs']['std'],disc_results['negatifs']['max'],disc_results['negatifs']['quant_3']))
print('Pour les scores neutres : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['neutre']['moyenne'],disc_results['neutre']['std'],disc_results['neutre']['max'],disc_results['neutre']['quant_3']))
print(' ')
print('#'*30)
print(' ')

#Sur la base No
disc_results=stats_scores(Base_SWN_no)

print(' ')
print(' Sur la bases des tweets No ({} tweets) : '.format(len(Base_SWN_no)))
print(' ')
print('Pour les scores positifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['positifs']['moyenne'],disc_results['positifs']['std'],disc_results['positifs']['max'],disc_results['positifs']['quant_3']))
print('Pour les scores negatifs : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['negatifs']['moyenne'],disc_results['negatifs']['std'],disc_results['negatifs']['max'],disc_results['negatifs']['quant_3']))
print('Pour les scores neutres : moyenne={}, ecart-type={}, max={}, quartile 3={}'.format(disc_results['neutre']['moyenne'],disc_results['neutre']['std'],disc_results['neutre']['max'],disc_results['neutre']['quant_3']))
print(' ')
print('#'*30)
print(' ')

In [ ]:
# Boxplot des distributions des scores positifs 

# Bases regroupant les scores positifs des deux bases 
lis_pos_no=[list(Base_SWN_no.Scores)[tweet][0] for tweet in range(len(list(Base_SWN_no.Scores)))]
lis_pos_yes=[list(Base_SWN_yes.Scores)[tweet][0] for tweet in range(len(list(Base_SWN_yes.Scores)))]

lis_neg_no=[list(Base_SWN_no.Scores)[tweet][1] for tweet in range(len(list(Base_SWN_no.Scores)))]
lis_neg_yes=[list(Base_SWN_yes.Scores)[tweet][1] for tweet in range(len(list(Base_SWN_yes.Scores)))]


Base_pos=pd.DataFrame([[x,y] for (x,y) in zip(lis_pos_yes,lis_pos_no)],columns=['Positifs Yes','Positifs No'])
Base_neg=pd.DataFrame([[x,y] for (x,y) in zip(lis_neg_yes,lis_neg_no)],columns=['Negatifs Yes','Negatifs No']) 

In [ ]:
x=Base_pos.plot.box(grid='True')

In [ ]:
x=Base_neg.plot.box(grid='True')

Etudier quels types de mots apparaissent des les plus forts quartiles

#### Clustering à partir des représentations 